# Imports

In [1]:
import openai
import pandas as pd
import json
import string
import gensim.downloader as api
import numpy as np
import time
from openai.embeddings_utils import pca_components_from_embeddings

In [2]:
def request_gpt(prompt: str = '', content: str = ''):
    """
    Wrapper that calls OpenAI's API to generate a response based on a system prompt and a body of text.

    system_prompt: str
    body: str

    return: str
    """
    _body = prompt + content
    res = openai.ChatCompletion.create(
        model='gpt-4',
        messages=[{'role': 'user', 'content': _body}]
    )
    return res.choices[0].message.content

In [18]:
# Load "credentials.json" which contains the value "open_ai_key"
openai.api_key = json.load(open("credentials.json"))["open_ai_key"]

In [4]:
# Load the data

corpus = pd.read_csv('CORPUS_ICPB.csv', index_col=0)
corpus

,modifyAt,createAt,titre,source,idArticle,texte,date,auteur,vecteur
0,NaN,2023-01-31 17:33:43+00:00,"Versailles: un ""expert en masculinité"" condamn...",BFMTV,0,"Des dossiers au tribunal de Versailles, dans l...",2023-01-25 16:43:00+00:00,AL,"[0.47458213567733765, -0.189848855137825, -0.0..."
1,NaN,2023-01-31 17:33:43+00:00,Finistère: un homme agresse un policier en lui...,BFMTV,1,Deux policiers ont été violemment agressés par...,2023-01-25 16:02:00+00:00,Justine Chevalier,"[0.17718319594860077, -0.5617039799690247, -0...."
2,NaN,2023-01-31 17:33:43+00:00,Grève du 31 janvier : à quelles perturbations ...,LePoint.fr,2,"La première journée de grève, jeudi 19 janvier...",2023-01-25 14:46:00+00:00,Par Thibaut Déléaz,"[-0.2377959042787552, -0.08118520677089691, 0...."
3,NaN,2023-01-31 17:33:43+00:00,"Puy-de-Dôme: une jeune femme tuée par balle, s...",BFMTV,3,(Voiture de police française) - Bertrand GUAY ...,2023-01-25 14:36:00+00:00,Mélanie Vecchio avec AL,"[0.24410541355609894, -0.2210969179868698, -0...."
4,NaN,2023-01-31 17:33:43+00:00,Le gouvernement demande le retrait de toutes l...,BFMTV,4,Couloir d'hôpital. (Photo d'illustration) - JU...,2023-01-25 14:30:00+00:00,J.F.,"[0.3187011480331421, 0.20401036739349365, 0.06..."
...,...,...,...,...,...,...,...,...,...
32315,NaN,2023-04-11 14:53:18+00:00,"Casinos Barrière : Alexandre Barrière, fils du...",LePoint.fr,16173,"Alexandre Barrière, âgé de 36 ans, est le fils...",2023-04-11 14:51:17+00:00,Par L.L avec AFP,NaN
32316,NaN,2023-04-11 14:53:18+00:00,Enseignants : une revalorisation salariale « e...,Journal du Dimanche,16174,"Le ministre de l’Éducation nationale, Pap Ndia...",2023-04-11 14:51:36+00:00,leJDD,NaN
32317,NaN,2023-04-11 14:53:18+00:00,Affaire Crédit lyonnais : Bernard Tapie aurait...,Journal du Dimanche,16175,Le Consortium de réalisation (CDR) commence à ...,2023-04-11 14:51:57+00:00,leJDD,NaN
32318,NaN,2023-04-11 14:53:18+00:00,Finistère : Prison ferme pour des Bosniens qui...,20minutes.fr,16176,ARNAQUE - Une centaine de victimes ont été rec...,2023-04-11 14:52:51+00:00,J.G. avec AFP,NaN


# Pre-processing

In [5]:
### Pre-processing of the data

# Drop some columns (modifyAt, createAt, idArticle, vecteur) which cannot be used
df = corpus[["titre", "source", "texte", "date", "auteur"]]

# Drop duplicated rows
df = df.drop_duplicates()

# Remove NaN values (In this case we remove a row if any NaN in it)
df = df.dropna()

# Create the column content which is the aggregation of title and text
df["content"] = "Titre: " + df["titre"]+ "\nContenu: " + df["texte"]

# Reset index
df.reset_index(drop=True, inplace=True)

# Save
df.to_csv('dataset_preprocessed.csv', index=False)

# Financial data filter

In [20]:
# Setup
FILTER_PROMPT = """

Write 1 if the following article is about finance or can impact the stock market, else 0:

--

"""

i_finance = 0 # Index of the row to process
df['is_finance'] = None # Creation of a new column which is True if related to Finance, else False
n_limit_finance = 1000 # Number of rows 

In [194]:
# Creation of a new column which is True if related to Finance, else False
while i_finance<n_limit_finance and i_finance<len(df):
    try:
        print("Request on the {}th row".format(i_finance))
        content = df.content.iloc[i_finance]
        res = request_gpt(prompt=FILTER_PROMPT, content=content)
        _bool = bool(int(res))
        df['is_finance'][i_finance] = _bool
        i_finance+=1
    except Exception as e:
        print("Something went wrong:\n\n" + str(e))
        time.sleep(2)

Request on the 901th row
Request on the 902th row
Request on the 903th row
Request on the 904th row
Request on the 905th row
Request on the 906th row
Request on the 907th row
Request on the 908th row
Request on the 909th row
Request on the 910th row


KeyboardInterrupt: 

In [195]:
df_finance = df[df.is_finance == True].reset_index(drop=True)
df_finance.to_csv("dataset_finance.csv", index=False)
df_finance

,titre,source,texte,date,auteur,content,is_finance
0,Grève du 31 janvier : à quelles perturbations ...,LePoint.fr,"La première journée de grève, jeudi 19 janvier...",2023-01-25 14:46:00+00:00,Par Thibaut Déléaz,Titre: Grève du 31 janvier : à quelles perturb...,True
1,"Chômage : une baisse de 3,6 % au quatrième tri...",LePoint.fr,"Le nombre de chômeurs en France s'élève à à 3,...",2023-01-25 13:24:00+00:00,Source AFP,"Titre: Chômage : une baisse de 3,6 % au quatri...",True
2,Le Vatican définit l’investissement chrétien e...,Challenges,"LES MARCHÉS AUJOURD'HUI\nPX1\n▼ -0,54%\nPX4\n▼...",2022-05-12 17:21:08+00:00,Article de Pierre,Titre: Le Vatican définit l’investissement chr...,True
3,Surtaxe d’habitation : découvrez les villes qu...,Capital,"LES MARCHÉS AUJOURD'HUI\nPX1\n▼ -1,44%\nPX4\n▼...",2022-12-16 16:36:45+00:00,Article de Jean,Titre: Surtaxe d’habitation : découvrez les vi...,True
4,Intergénérationnel : Focus sur les nombreux pr...,Zinfos,"LES MARCHÉS AUJOURD'HUI\nPX1\n▲ +0,60%\nPX4\n▲...",2022-12-19 16:40:40+00:00,Article de Ville de Saint,Titre: Intergénérationnel : Focus sur les nomb...,True
...,...,...,...,...,...,...,...
382,"Assemblée nationale : Cyrielle Chatelain, l’in...",20 Minutes,La jeune présidente du groupe écologiste à l’A...,2023-02-19 12:25:00+00:00,Rachel Garrat Valcarcel,Titre: Assemblée nationale : Cyrielle Chatelai...,True
383,L'Ukraine tente de réparer ses sites énergétiq...,Euronews français,Les autorités ukrainiennes tentent d'atténuer ...,2023-02-20 08:47:32+00:00,Euronews,Titre: L'Ukraine tente de réparer ses sites én...,True
384,"Avec la flambée des prix de l'énergie, le chif...",La Tribune,L'avenir s'annonce radieux pour Air Liquide qu...,2023-02-20 08:47:32+00:00,latribune.fr,"Titre: Avec la flambée des prix de l'énergie, ...",True
385,Guerre en Ukraine : des démocrates américains ...,L'Express,Le président américain fait face à des critiqu...,2023-02-20 08:47:32+00:00,lexpress.fr,Titre: Guerre en Ukraine : des démocrates amér...,True


# Extraction of variables

In [196]:
# Setup
EXTRACTION_PROMPT = """

Votre tâche consiste à trouver des variables pertinentes dans un article de finance. Essayez de donner des noms de variables génériques, qui seront également identifiables dans d'autres textes.
J'aimerais que vous identifiiez une tendance dans l'article qui sera la variable résultat, que nous appellerons "Indicateur". Cet "Indicateur" sera l'un des trois termes suivants : "POSITIF", "NEUTRE" ou "NEGATIF", selon le sentiment de l'article sur les cours et indices boursiers, les tendances du marché, etc.

Par exemple, si vous voyez :
Lorsqu'une entreprise connaît une baisse importante de ses bénéfices ou une réduction de son activité, elle peut être contrainte de réduire ses effectifs en procédant à des licenciements ou en gelant les embauches, ce qui peut contribuer à une augmentation du taux de chômage dans la région où elle est implantée. 
Nous vous demandons d'extraire : la situation financière de l'entreprise, les décisions de l'entreprise, le taux de chômage, le cours de l'action de l'entreprise.

Vous ne devez renvoyer qu'une chaîne de caractères au format JSON suivant :
{
    "Indicateur" : "valeur 0",
    "Situation Financière de l'Entreprise" : "valeur 1",
    "Décisions de l'Entreprise" : "valeur 2",
    ...
}
où les valeurs sont les mots trouvés dans l'article en rapport avec la variable extraite.

Voici l'article à analyser :

---

"""

i_extraction = 0 # Index of the row to process
df_finance['extraction'] = None # Creation of a new column "extraction" which contains a dict. keys are variables and values are text from the article related to the variable

n_limit_extraction = 400

In [106]:
while i_extraction<n_limit_extraction and i_extraction<len(df_finance):
    try:
        print("Request on the {}th row".format(i_extraction))
        content = df_finance.content.iloc[i_extraction]
        res = request_gpt(prompt=EXTRACTION_PROMPT, content=content)
        _json = json.loads(res.replace("\n", ""))
        df_finance['extraction'][i_extraction] = _json
        i_extraction+=1
    except Exception as e:
        print("Something went wrong:\n\n" + str(e))
        time.sleep(5)

In [107]:
df_finance.to_csv("dataset_extract.csv", index=False)
df_finance

,titre,source,texte,date,auteur,content,is_finance,extraction
0,Grève du 31 janvier : à quelles perturbations ...,LePoint.fr,"La première journée de grève, jeudi 19 janvier...",2023-01-25 14:46:00+00:00,Par Thibaut Déléaz,Titre: Grève du 31 janvier : à quelles perturb...,True,"{'Indicateur': 'NEGATIF', 'Situation Politique..."
1,"Chômage : une baisse de 3,6 % au quatrième tri...",LePoint.fr,"Le nombre de chômeurs en France s'élève à à 3,...",2023-01-25 13:24:00+00:00,Source AFP,"Titre: Chômage : une baisse de 3,6 % au quatri...",True,"{'Indicateur': 'POSITIF', 'Taux de Chômage': '..."
2,Le Vatican définit l’investissement chrétien e...,Challenges,"LES MARCHÉS AUJOURD'HUI\nPX1\n▼ -0,54%\nPX4\n▼...",2022-05-12 17:21:08+00:00,Article de Pierre,Titre: Le Vatican définit l’investissement chr...,True,"{'Indicateur': 'NEUTRE', 'Normes d'Investissem..."
3,Surtaxe d’habitation : découvrez les villes qu...,Capital,"LES MARCHÉS AUJOURD'HUI\nPX1\n▼ -1,44%\nPX4\n▼...",2022-12-16 16:36:45+00:00,Article de Jean,Titre: Surtaxe d’habitation : découvrez les vi...,True,"{'Indicateur': 'NEUTRE', 'Situation Financière..."
4,Intergénérationnel : Focus sur les nombreux pr...,Zinfos,"LES MARCHÉS AUJOURD'HUI\nPX1\n▲ +0,60%\nPX4\n▲...",2022-12-19 16:40:40+00:00,Article de Ville de Saint,Titre: Intergénérationnel : Focus sur les nomb...,True,"{'Indicateur': 'POSITIF', 'Performance des Mar..."
...,...,...,...,...,...,...,...,...
382,"Assemblée nationale : Cyrielle Chatelain, l’in...",20 Minutes,La jeune présidente du groupe écologiste à l’A...,2023-02-19 12:25:00+00:00,Rachel Garrat Valcarcel,Titre: Assemblée nationale : Cyrielle Chatelai...,True,"{'Indicateur': 'NEUTRE', 'Rôle des Acteurs Pol..."
383,L'Ukraine tente de réparer ses sites énergétiq...,Euronews français,Les autorités ukrainiennes tentent d'atténuer ...,2023-02-20 08:47:32+00:00,Euronews,Titre: L'Ukraine tente de réparer ses sites én...,True,"{'Indicateur': 'NEGATIF', 'Situation Politique..."
384,"Avec la flambée des prix de l'énergie, le chif...",La Tribune,L'avenir s'annonce radieux pour Air Liquide qu...,2023-02-20 08:47:32+00:00,latribune.fr,"Titre: Avec la flambée des prix de l'énergie, ...",True,"{'Indicateur': 'POSITIF', 'Situation Financièr..."
385,Guerre en Ukraine : des démocrates américains ...,L'Express,Le président américain fait face à des critiqu...,2023-02-20 08:47:32+00:00,lexpress.fr,Titre: Guerre en Ukraine : des démocrates amér...,True,"{'Indicateur': 'NEUTRE', 'Situation Politique'..."


# Reducing the number of variables

In [108]:
# Creation of a ChatGPT request which takes the list of all extracted variables and create a dict with more global variables as key and the extracted variables as values

# Setup
REDUCTION_PROMPT = """

Je vais te donner une liste de variables. Regroupe les variables en groupes. Les variables peuvent être représentées dans un ou plusieurs groupes.
N'oublie aucune variable. Je veux que toutes les variables soient représentées dans des catégories, tu ne dois pas en laisser de côté.

---

Par exemple, si je te donne "Manifestation", "Accident", "Taux de pauvreté", "Taxe carbone"...
Tu dois créer le dictionnaire suivant :
{
    "Social" : ["Manifestation", "Accident", "Taux de pauvreté",...],
    "Economique" : ["Taux de pauvreté",...],
    "Travail" : ["Manifestation"],
    "Entreprise" : ["Taxe carbone",...]
    "Environnement" : ["Taxe carbone",...]
    ...
}

---

"Indicateur" est une catégorie. Essaie de regrouper la liste en moins de 10 catégories. Envoie uniquement le dictionnaire.

Ci-dessous la liste des variables :

"""

In [109]:
# Creation of the list of all variables
variables_list = []
for row in df_finance.iterrows():
    try:
        _dict = row[1].extraction.copy()
        _dict.pop("Indicateur")
        variables_list+= list(_dict.keys())
    except:
        pass
variables_list

['Situation Politique',
 'Décisions Syndicales',
 'Mobilisation Citoyenne',
 'Répercussions sur les Transports',
 'Prévisions Syndicales',
 'Taux de Chômage',
 'Nombre de Chômeurs',
 'Changement par rapport au Trimestre Précédent',
 'Comparaison Historique',
 "Nombre de Demandeur d'Emploi (incluant activité réduite)",
 "Normes d'Investissement",
 'Décisions de la Commission Européenne',
 'Montant des Investissements',
 'Publication du Vatican',
 'Valeur du Document Vatican',
 "Situation Financière de l'Entreprise",
 "Décisions de l'Communes",
 'Marché Immobilier',
 'Réglementations Financières',
 "Taux d'Impôt",
 'Performance des Marchés',
 'Politique de la Commune',
 'Projets de la Commune',
 'Décision Politique',
 'Crise Alimentaire',
 'Opposition Politique',
 'Climat',
 'Situation du Pays',
 'Résultats Élections',
 'Situation Politique',
 'Intentions de vote',
 'Impact événements externes',
 'Soutiens politiques',
 'Situation Politique et Sociale du Pays',
 'Actions du Gouvernement'

In [ ]:
# Send a request to ChatGPT in order to create the dict
content = "["+"".join(x + ", "for x in variables_list)+"]"
res = request_gpt(prompt=REDUCTION_PROMPT, content=content)
variables_dict = json.loads(res.replace("\n", ""))

In [113]:
# Inversed dict
variables_dict_inversed = {}

for variable, liste in variables_dict.items():
    for e in liste:
        if e not in variables_dict_inversed:
            variables_dict_inversed[e] = [variable]
        else:
            variables_dict_inversed[e].append(variable)
variables_dict_inversed["Indicateur"] = ["Indicateur"]

In [114]:
# Check if every variable has been mapped
not_mapped = []
for row in df_finance.iterrows():
    _dict = row[1].extraction
    for key, value in _dict.items():
        if key not in variables_dict_inversed:
            print("This key has not been mapped: " + key)
            not_mapped.append(key)

# If not remap
# Lost of information, so try to ask GPT to add to the categories or do it manually
# Need to improve this part

This key has not been mapped: Evénements Climatiques Extrêmes
This key has not been mapped: Attaques Militaires
This key has not been mapped: Paroles d'un Responsable Politique
This key has not been mapped: Image d'Entreprise
This key has not been mapped: Conséquences d'Entreprise
This key has not been mapped: Opinions Publiques
This key has not been mapped: Evènements Politiques
This key has not been mapped: Evènements Ecologiques
This key has not been mapped: Production Energétique
This key has not been mapped: Mouvement Contestataire
This key has not been mapped: Impact sur la Science
This key has not been mapped: Situation politique et militaire
This key has not been mapped: Rôle des civils
This key has not been mapped: Conséquences du conflit
This key has not been mapped: Objectifs de la guerre
This key has not been mapped: Enjeux environnementaux
This key has not been mapped: Investissements et moyens alloués
This key has not been mapped: Impacts et conséquences
This key has not 

In [71]:
# Remapping if some variables not mapped
categories = list(variables_dict.keys())[1:]

dict_example = """

{
    "Public Reaction" : ["Manifestation", "Accident", "Taux de pauvreté"],
    "Economy" : ["Taux de pauvreté"],
    ...
}

"""

MAPPING_PROMPT = """ 

Je vais te donner une liste de variables. Une variable peut être affectée à une ou plusieurs catégories. J'aimerais que tu les regroupes dans les catégories suivantes.  :

Catégories: {categories}

---

Par exemple, si je te donne "Manifestation", "Accident", "Taux de pauvreté"...
Tu dois créer le dictionnaire suivant :

{example}

---

Voici la liste des variables :

Variables : {variables}


""".format(categories = "["+"".join(x + ", "for x in categories)+"]",variables = "["+"".join(x + ", "for x in not_mapped)+"]", example = dict_example)

res = request_gpt(MAPPING_PROMPT, "")



In [115]:
# Creation of the new dataframe where df[i,j] are the text from article i related to variable j
df_variables = pd.DataFrame()
for row in df_finance.iterrows():
    _dict = row[1].extraction
    new_dict = {}
    for key, value in _dict.items():
        if type(value)==list:
            value = " ".join(value)
        if value is None:
            value=""
        # Concatenate global key with the concatenation of values
        if key in variables_dict_inversed:
            for category in variables_dict_inversed[key]:
                if category not in new_dict:
                    new_dict[category] = value
                else:
                    new_dict[category] = new_dict[category] + " "+ value
        else:
            pass                 
    df_variables = pd.concat([df_variables, pd.DataFrame({row[0]: new_dict}).transpose()])

df_variables.to_csv("dataset_variable.csv", index=False)
df_variables

,Indicateur,Politique,Économique,Economique,Changement Climatique,Social
0,NEGATIF,projet de réforme des retraites du gouvernemen...,entre un TGV sur trois et un TGV sur cinq selo...,NaN,NaN,NaN
1,POSITIF,NaN,NaN,"baisse 3,6 % quatrième trimestre 3,050 million...",NaN,NaN
2,NEUTRE,Directive sur la publication d'informations en...,NaN,"Vatican, Investissement responsable, liste de ...",NaN,NaN
3,NEUTRE,N/A majoration de la taxe d’habitation sur les...,NaN,"N/A N/A N/A N/A hausse du marché immobilier, d...",NaN,NaN
4,POSITIF,"Roxanne PAUSÉ DAMOUR, déléguée à la politique ...",NaN,"PX1 ▲ +0,60%, PX4 ▲ +0,60%, CACAS ▲ +0,39%, EU...",NaN,NaN
...,...,...,...,...,...,...
382,NEUTRE,NaN,NaN,NaN,NaN,NaN
383,NEGATIF,Guerre Ukraine-Russie Guerre Ukraine-Russie Gu...,NaN,NaN,NaN,NaN
384,POSITIF,"chiffre d'affaires envolé de 41,3%, ventes pro...","confirme ses objectifs annuels, augmenter à no...","chiffre d'affaires envolé de 41,3%, ventes pro...",NaN,NaN
385,NEUTRE,"Guerre en Ukraine, Critiques de démocrates amé...",NaN,"Rejet des allégations russes par l'Otan, Fusti...",NaN,NaN


# Encoding the values

In [116]:
# Function which transform "POSITIF" in 1, "NEUTRE" in 0, "NEGATIF" in "-1" 
def indicateur_transform(s):
    if s.lower()=="negatif":
        return -1
    if s.lower()=="positif":
        return 1
    return 0

# Function which use the openai API to encode a string 
def embed(string: str):
    if string=="":
        return np.NaN
    response = openai.Embedding.create(
        input=string,
        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']
    return embeddings

# encoding of every value in the dataframe
df_variables_embed = df_variables.copy().drop(columns="Indicateur")
df_variables_embed.fillna("", inplace=True)
df_variables_embed = df_variables_embed.applymap(embed)
df_variables_embed["Indicateur"] = df_variables["Indicateur"].apply(indicateur_transform)
df_variables_embed.to_csv('dataset_variable_embeded.csv', index=False)

df_variables_embed

KeyboardInterrupt: 

# PCA reduction

In [ ]:
# Transform vectors into reels: use of a PCA on each column but the "Indicateur" column which is already numeric
df_pca = df_variables_embed.copy()
for col in df_variables_embed.columns.drop("Indicateur"):
    _series = df_pca[col]
    _list_without_nan = list(_series[~_series.isna()])
    pca_coeff = pca_components_from_embeddings(_list_without_nan, 1)
    _series[~_series.isna()] = pca_coeff

df_pca.fillna(0, inplace=True)


col_abssum = df_pca.abs().sum()
df_pca = df_pca.loc[:,~(col_abssum==0)]
df_pca["Indicateur"] = df_variables["Indicateur"]
df_pca.to_csv("dataset_pca.csv", index=False)